In [156]:
import pandas as pd
data=pd.read_csv('StackOverflow_questions_2009.csv')

In [157]:
import pandas as pd
import numpy as np
import re
import nltk 
import spacy
from tqdm import tqdm 
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split

%matplotlib inline 
pd.set_option('display.max_colwidth', 200)
import tensorflow as tf
from tensorflow import keras 

In [158]:
data.head()

,Id,CreationDate,Title,Body,Tags,ViewCount,CommentCount,AnswerCount,Score
0,414336,2009-01-05 20:09:49,Why does the SqlServer optimizer get so confused with parameters?,"<p>I know this has something to do with parameter sniffing, but I'm just perplexed at how something like the following example is even possible with a piece of technology that does so many complex...",<sql><sql-server><optimization>,3368,6,9,7
1,414605,2009-01-05 21:33:21,Fluent interface for rendering HTML,"<p>Rendering HTML with the HtmlTextWriter isn't incredibly intuitive in my opinion, but if you're implementing web controls in web forms it's what you have to work with. I thought that it might be...",<.net><html><render><web-controls><htmltextwriter>,4557,7,4,10
2,414714,2009-01-05 22:21:04,Compiling with g++ using multiple cores,<p>Quick question: what is the compiler flag to allow g++ to spawn multiple instances of itself in order to compile large projects quicker (for example 4 source files at a time for a multi-core CP...,<c++><compiler-construction><makefile><g++><multicore>,159848,7,8,181
3,415452,2009-01-06 04:26:16,Object-orientation in C,<p>What would be a set of nifty preprocessor hacks (ANSI C89/ISO C90 compatible) which enable some kind of ugly (but usable) object-orientation in C?</p>\n\n<p>I am familiar with a few different o...,<c><oop><object>,82624,6,23,161
4,415515,2009-01-06 04:55:55,How can I read and manipulate CSV file data in C++?,"<p>Pretty self-explanatory, I tried google and got a lot of the dreaded expertsexchange, I searched here as well to no avail. An online tutorial or example would be best. Thanks guys.</p>\n",<c++><csv>,147860,7,9,51


In [159]:
data.shape

(3931, 9)

In [160]:
data = data[['Title', 'Body', 'Tags']]
data["text"] = data["Title"] + " " + data["Body"]
data.head()

,Title,Body,Tags,text
0,Why does the SqlServer optimizer get so confused with parameters?,"<p>I know this has something to do with parameter sniffing, but I'm just perplexed at how something like the following example is even possible with a piece of technology that does so many complex...",<sql><sql-server><optimization>,"Why does the SqlServer optimizer get so confused with parameters? <p>I know this has something to do with parameter sniffing, but I'm just perplexed at how something like the following example is ..."
1,Fluent interface for rendering HTML,"<p>Rendering HTML with the HtmlTextWriter isn't incredibly intuitive in my opinion, but if you're implementing web controls in web forms it's what you have to work with. I thought that it might be...",<.net><html><render><web-controls><htmltextwriter>,"Fluent interface for rendering HTML <p>Rendering HTML with the HtmlTextWriter isn't incredibly intuitive in my opinion, but if you're implementing web controls in web forms it's what you have to w..."
2,Compiling with g++ using multiple cores,<p>Quick question: what is the compiler flag to allow g++ to spawn multiple instances of itself in order to compile large projects quicker (for example 4 source files at a time for a multi-core CP...,<c++><compiler-construction><makefile><g++><multicore>,Compiling with g++ using multiple cores <p>Quick question: what is the compiler flag to allow g++ to spawn multiple instances of itself in order to compile large projects quicker (for example 4 so...
3,Object-orientation in C,<p>What would be a set of nifty preprocessor hacks (ANSI C89/ISO C90 compatible) which enable some kind of ugly (but usable) object-orientation in C?</p>\n\n<p>I am familiar with a few different o...,<c><oop><object>,Object-orientation in C <p>What would be a set of nifty preprocessor hacks (ANSI C89/ISO C90 compatible) which enable some kind of ugly (but usable) object-orientation in C?</p>\n\n<p>I am familia...
4,How can I read and manipulate CSV file data in C++?,"<p>Pretty self-explanatory, I tried google and got a lot of the dreaded expertsexchange, I searched here as well to no avail. An online tutorial or example would be best. Thanks guys.</p>\n",<c++><csv>,"How can I read and manipulate CSV file data in C++? <p>Pretty self-explanatory, I tried google and got a lot of the dreaded expertsexchange, I searched here as well to no avail. An online tutorial..."


In [161]:
import re
def clean_text(text):
    # removing the tags and urls 
    text= re.sub(r'<.*?>', "", text)
    #keeping only the alphabets
    text=re.sub(r'^[a-zA-z]', " ", text)
    # removing white spaces
    text=' '.join(text.split())
    return text

data["text"] = data["text"].apply(lambda x:clean_text(x))
data["text"]= data["text"].str.lower()
data["text"].head()

0    hy does the sqlserver optimizer get so confused with parameters? i know this has something to do with parameter sniffing, but i'm just perplexed at how something like the following example is even...
1    luent interface for rendering html rendering html with the htmltextwriter isn't incredibly intuitive in my opinion, but if you're implementing web controls in web forms it's what you have to work ...
2    ompiling with g++ using multiple cores quick question: what is the compiler flag to allow g++ to spawn multiple instances of itself in order to compile large projects quicker (for example 4 source...
3    bject-orientation in c what would be a set of nifty preprocessor hacks (ansi c89/iso c90 compatible) which enable some kind of ugly (but usable) object-orientation in c? i am familiar with a few d...
4    ow can i read and manipulate csv file data in c++? pretty self-explanatory, i tried google and got a lot of the dreaded expertsexchange, i searched here as well to no avail. a

Removing Stopwords from the Data 

In [162]:
from nltk.corpus import stopwords 
stop_words=set(stopwords.words('english'))
def strip_stopwords(text):
    words= [w for w in text.split() if w not in stop_words]
    text_clean= " ".join(words)
    return text_clean
data["text"] = data["text"].apply(lambda x: strip_stopwords(x))
data.head()

,Title,Body,Tags,text
0,Why does the SqlServer optimizer get so confused with parameters?,"<p>I know this has something to do with parameter sniffing, but I'm just perplexed at how something like the following example is even possible with a piece of technology that does so many complex...",<sql><sql-server><optimization>,"hy sqlserver optimizer get confused parameters? know something parameter sniffing, i'm perplexed something like following example even possible piece technology many complex things well. many us r..."
1,Fluent interface for rendering HTML,"<p>Rendering HTML with the HtmlTextWriter isn't incredibly intuitive in my opinion, but if you're implementing web controls in web forms it's what you have to work with. I thought that it might be...",<.net><html><render><web-controls><htmltextwriter>,"luent interface rendering html rendering html htmltextwriter incredibly intuitive opinion, implementing web controls web forms work with. thought might possible create fluent interface reads bit l..."
2,Compiling with g++ using multiple cores,<p>Quick question: what is the compiler flag to allow g++ to spawn multiple instances of itself in order to compile large projects quicker (for example 4 source files at a time for a multi-core CP...,<c++><compiler-construction><makefile><g++><multicore>,ompiling g++ using multiple cores quick question: compiler flag allow g++ spawn multiple instances order compile large projects quicker (for example 4 source files time multi-core cpu)?
3,Object-orientation in C,<p>What would be a set of nifty preprocessor hacks (ANSI C89/ISO C90 compatible) which enable some kind of ugly (but usable) object-orientation in C?</p>\n\n<p>I am familiar with a few different o...,<c><oop><object>,"bject-orientation c would set nifty preprocessor hacks (ansi c89/iso c90 compatible) enable kind ugly (but usable) object-orientation c? familiar different object-oriented languages, please respon..."
4,How can I read and manipulate CSV file data in C++?,"<p>Pretty self-explanatory, I tried google and got a lot of the dreaded expertsexchange, I searched here as well to no avail. An online tutorial or example would be best. Thanks guys.</p>\n",<c++><csv>,"ow read manipulate csv file data c++? pretty self-explanatory, tried google got lot dreaded expertsexchange, searched well avail. online tutorial example would best. thanks guys."


In [163]:
from sklearn.preprocessing import MultiLabelBinarizer
multi_label= MultiLabelBinarizer()
multi_label.fit(data["Tags"])
Y= multi_label.transform(data["Tags"])
Y.shape
(76365, 100)



(76365, 100)

In [164]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

tokenizer = Tokenizer()
tokenizer.fit_on_texts(data['text'])
# check unique words count
len(tokenizer.word_index)
81956
# check unique words count
vocab_size = len(tokenizer.word_index) + 1
vocab_size
81957
sequences = tokenizer.texts_to_sequences(data['text'])
i = 0
print(data['text'][i], '\n'), print(sequences[i])

hy sqlserver optimizer get confused parameters? know something parameter sniffing, i'm perplexed something like following example even possible piece technology many complex things well. many us run stored procedures intermittently run several orders magnitude slower usual, copy sql procedure use parameter values separate query window, runs fast usual. fixed procedure like converting this: alter procedure p_myproc ( @param1 int ) -- complex query @param1 this: alter procedure p_myproc ( @param1 int ) declare @param1copy int; set @param1copy = @param1; -- query using @param1copy went running minute back one second, like usually runs. behavior seems totally random. 9 10 @param1 inputs, query fast, regardless much data ends needing crunch, big result set it. 1 10, gets lost. fix replace int int query? makes sense. [edit] @gbn linked question, details similar problem: known issue?: sql server 2005 stored procedure fails complete parameter hesitate cry "bug!" that's often cop-out, really se

(None, None)

In [165]:
max_length = 125

# padding
padded_seq = pad_sequences(sequences, maxlen=max_length)
from sklearn.preprocessing import MultiLabelBinarizer

multilabel_binarizer = MultiLabelBinarizer()
multilabel_binarizer.fit(data['Tags'])
y = multilabel_binarizer.transform(data['Tags'])
padded_seq.shape, y.shape
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(padded_seq, y, 
                                                    test_size=0.2, 
                                                    random_state=9)
from keras.models import Sequential, load_model
from keras.layers import Dense, Embedding, GlobalMaxPool1D, Dropout, Conv1D
from keras.callbacks import EarlyStopping, ModelCheckpoint
from sklearn.model_selection import GridSearchCV


In [166]:
model = Sequential()
model.add(Embedding(vocab_size +1, 128, input_length = max_length))
model.add(Dropout(0.15))
model.add(Conv1D(300, 5, padding = 'valid', activation = "relu", strides = 1))
model.add(GlobalMaxPool1D())
model.add(Dense(42, activation = "sigmoid"))
model.compile(optimizer='adam', 
              loss='binary_crossentropy', 
              metrics=['accuracy'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 125, 128)          3808384   
                                                                 
 dropout_2 (Dropout)         (None, 125, 128)          0         
                                                                 
 conv1d_2 (Conv1D)           (None, 121, 300)          192300    
                                                                 
 global_max_pooling1d_2 (Gl  (None, 300)               0         
 obalMaxPooling1D)                                               
                                                                 
 dense_2 (Dense)             (None, 42)                12642     
                                                                 
Total params: 4013326 (15.31 MB)
Trainable params: 4013326 (15.31 MB)
Non-trainable params: 0 (0.00 Byte)
______________

In [167]:
callbacks = [
             EarlyStopping(patience=3),
             ModelCheckpoint(filepath='model-conv1d_v1.h5', save_best_only=True)
            ]

# train model
history = model.fit(x_train, y_train,
                    epochs=15,
                    batch_size=128,
                    validation_split=0.1,
                    callbacks=callbacks)

Epoch 1/15
23/23 [==============================] - 5s 180ms/step - loss: 0.5845 - accuracy: 0.0103 - val_loss: 0.4638 - val_accuracy: 0.0762
Epoch 2/15


c:\Python311\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


23/23 [==============================] - 5s 201ms/step - loss: 0.4256 - accuracy: 0.0428 - val_loss: 0.4096 - val_accuracy: 0.0190
Epoch 3/15
23/23 [==============================] - 5s 233ms/step - loss: 0.4015 - accuracy: 0.0148 - val_loss: 0.4028 - val_accuracy: 0.0254
Epoch 4/15
23/23 [==============================] - 5s 198ms/step - loss: 0.3961 - accuracy: 0.0308 - val_loss: 0.3993 - val_accuracy: 0.0603
Epoch 5/15
23/23 [==============================] - 4s 191ms/step - loss: 0.3901 - accuracy: 0.0703 - val_loss: 0.3962 - val_accuracy: 0.0921
Epoch 6/15
23/23 [==============================] - 4s 194ms/step - loss: 0.3806 - accuracy: 0.1131 - val_loss: 0.3908 - val_accuracy: 0.2000
Epoch 7/15
23/23 [==============================] - 5s 199ms/step - loss: 0.3671 - accuracy: 0.1813 - val_loss: 0.3847 - val_accuracy: 0.2603
Epoch 8/15
23/23 [==============================] - 5s 207ms/step - loss: 0.3514 - accuracy: 0.2333 - val_loss: 0.3818 - val_accuracy: 0.3111
Epoch 9/15
23/23 

### TFIDF 
- Reason to use TFIDF and not lemmetization is beacause we don't want the semantice meaning in this case 


In [168]:
from sklearn.feature_extraction.text import TfidfVectorizer
tfidf= TfidfVectorizer(max_df=0.8, max_features=1000)

X_tf = tfidf.fit_transform(data["text"])
X_tf.shape
#Tfidf.shape

(3931, 1000)

In [169]:
X_train_tf, X_test_tf, y_train_tf, y_test_tf = train_test_split(X_tf, Y, test_size=0.2, random_state=42)

In [170]:
# from sklearn.metrics import accuracy_score
# from sklearn.multiclass import OneVsRestClassifier
# from sklearn.svm import SVC

# # Assuming X_train_tf and y_train_tf are your training data
# classifier = OneVsRestClassifier(SVC(kernel='linear'))
# classifier.fit(X_train_tf, y_train_tf)

# y_pred = classifier.predict(X_test_tf)
# accuracy = accuracy_score(y_test_tf, y_pred)
# print("Accuracy:", accuracy)

So I tried these algorithms for the above cases and got bad results 
- Multinomial Naive Bayes 
- OneVsRestClassifier with Support Vector Machines (SVM)
- OneVsRestClassifier with Logistic Regression
- OneVsRestClassifier with Random Forest

### Using Word2vec

In [171]:
from nltk.corpus import stopwords 
stop_words=set(stopwords.words('english'))
def strip_stopwords(text):
    words= [w for w in text.split() if w not in stop_words]
    text_clean= " ".join(words)
    return text_clean
data["text"] = data["text"].apply(lambda x: strip_stopwords(x))
data.head()

,Title,Body,Tags,text
0,Why does the SqlServer optimizer get so confused with parameters?,"<p>I know this has something to do with parameter sniffing, but I'm just perplexed at how something like the following example is even possible with a piece of technology that does so many complex...",<sql><sql-server><optimization>,"hy sqlserver optimizer get confused parameters? know something parameter sniffing, i'm perplexed something like following example even possible piece technology many complex things well. many us r..."
1,Fluent interface for rendering HTML,"<p>Rendering HTML with the HtmlTextWriter isn't incredibly intuitive in my opinion, but if you're implementing web controls in web forms it's what you have to work with. I thought that it might be...",<.net><html><render><web-controls><htmltextwriter>,"luent interface rendering html rendering html htmltextwriter incredibly intuitive opinion, implementing web controls web forms work with. thought might possible create fluent interface reads bit l..."
2,Compiling with g++ using multiple cores,<p>Quick question: what is the compiler flag to allow g++ to spawn multiple instances of itself in order to compile large projects quicker (for example 4 source files at a time for a multi-core CP...,<c++><compiler-construction><makefile><g++><multicore>,ompiling g++ using multiple cores quick question: compiler flag allow g++ spawn multiple instances order compile large projects quicker (for example 4 source files time multi-core cpu)?
3,Object-orientation in C,<p>What would be a set of nifty preprocessor hacks (ANSI C89/ISO C90 compatible) which enable some kind of ugly (but usable) object-orientation in C?</p>\n\n<p>I am familiar with a few different o...,<c><oop><object>,"bject-orientation c would set nifty preprocessor hacks (ansi c89/iso c90 compatible) enable kind ugly (but usable) object-orientation c? familiar different object-oriented languages, please respon..."
4,How can I read and manipulate CSV file data in C++?,"<p>Pretty self-explanatory, I tried google and got a lot of the dreaded expertsexchange, I searched here as well to no avail. An online tutorial or example would be best. Thanks guys.</p>\n",<c++><csv>,"ow read manipulate csv file data c++? pretty self-explanatory, tried google got lot dreaded expertsexchange, searched well avail. online tutorial example would best. thanks guys."


In [172]:
from gensim.utils import simple_preprocess
X = data['text'].apply(lambda x: simple_preprocess(x))
from gensim.models import Word2Vec 
w2v_model = Word2Vec(X, min_count=2, sg=1, negative= 5)

In [173]:
# Assuming you have a Word2Vec model named w2v_model
words = list(w2v_model.wv.key_to_index.keys())[:20]
print(words)

for word in words:  # Iterate directly over the words
    print(f'{word}')  # Print the word without trying to unpack a count


['lt', 'gt', 'code', 'like', 'using', 'use', 'would', 'int', 'string', 'one', 'new', 'get', 'way', 'this', 'class', 'file', 'time', 'return', 'want', 'java']
lt
gt
code
like
using
use
would
int
string
one
new
get
way
this
class
file
time
return
want
java


In [174]:
words_statistics = w2v_model.wv.most_similar('statistics')
words_statistics 

[('upwards', 0.9541953206062317),
 ('locked', 0.951536238193512),
 ('violate', 0.948758602142334),
 ('screenshots', 0.9486634135246277),
 ('privileges', 0.9484642148017883),
 ('cracking', 0.9463973045349121),
 ('afraid', 0.9459027051925659),
 ('normalize', 0.9456357359886169),
 ('specifics', 0.9453726410865784),
 ('offered', 0.9452270269393921)]

In [175]:
import numpy as np
import pandas as pd

def get_embedding_w2v(doc_tokens):
    embeddings = []
    model = w2v_model
    # Iterate over tokens to extract their vectors
    for tok in doc_tokens:
        if tok in model.wv:  # Check for word in model directly
            embeddings.append(model.wv[tok])  # Get vector using model[tok]
    # Mean the vectors of individual words to get the vector of the statement
    return np.mean(embeddings, axis=0)

X_w2v_model = X.apply(lambda x: get_embedding_w2v(x))
X_w2v_model = pd.DataFrame(X_w2v_model.tolist())
print('X shape:', X_w2v_model.shape)


X shape: (3931, 100)


In [176]:
X_train_wm, X_test_wm, y_train_wm, y_test_wm = train_test_split(X_w2v_model, Y, test_size=0.2, random_state=42)
from sklearn.linear_model import LogisticRegression
#from sklearn.naive_bayes import GaussianNB

# Binary Relevance
from sklearn.multiclass import OneVsRestClassifier

# Performance metric
from sklearn.metrics import f1_score
lr = LogisticRegression()
clf = OneVsRestClassifier(lr)
clf.fit(X_train_wm, y_train_wm)

c:\Python311\Lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label 14 is present in all training examples.
  warnings.warn(
c:\Python311\Lib\site-packages\sklearn\multiclass.py:84: UserWarning: Label 15 is present in all training examples.
  warnings.warn(


OneVsRestClassifier(estimator=LogisticRegression())

In [177]:
y_pred_prob = clf.predict_proba(X_test_wm)
from sklearn.metrics import accuracy_score
# set threshold value
t = 0.45

# convert to integers
y = (y_pred_prob >= t).astype(int)
f1_score(y_test_wm, y, average="micro")

print("Accuracy:", accuracy_score(y_test_wm, y)*100, "%")

Accuracy: 0.0 %
